In [4]:
import pandas as pd
import yfinance as yf 
import ut_psql_connect 
import ut_update_queries as q
from dateutil.relativedelta import relativedelta
import datetime
import time
import numpy as np

import yahoo_fin.stock_info as si

##engine
engine=ut_psql_connect.engine
query_local = q.get_ticker_list (2) # default to all  KPI for consistants  select * from r1_update_versions 
#DF_TICKERS
DF_TICKERS = pd.read_sql_query(query_local, con=engine)
LS_TICKERS = list(DF_TICKERS['ticker'])
#list_of_chunks = list( q.chunks(LS_TICKERS, 10))
LS_TICKERS = list(('ABMD', 'ABT'))

######################################################
###Storage 
######################################################

##diccionaries dowload
Info_dict= {}
Price_dict= {}

##data  manipulations
DATAFRAME_COLLECTION = {}
Y_DICT_COLLECTION = {}   
DF = 'DF_YAHOO_RECOMMENDATIONS_IMP'
DF2 = 'DF_BUY_SELL_FLAG_IMP_TEST'
DF_info= pd.DataFrame()
DF_close = pd.DataFrame()
DF_YAHOO_RECOMMENDATIONS_IMP = pd.DataFrame()
DF_BUY_SELL_FLAG_IMP_TEST= pd.DataFrame()

# The keys you want and name to convert to 
columns_rename={ 'index' : 'ticker'
                ,'shortName' : 'name'
                , 'marketCap' :'market_cap' 
   
                 ,'targetMeanPrice' : 'y_price_1y_est' 
                ,'recommendationMean' : 'y_rank' 
                ,'numberOfAnalystOpinions' : 'y_rank_Analyst_num' 
                ,'profitMargins' : 'profit_Margins'
              
                ,'trailingEps': 'EPS'
                ,'trailingPE': 'PE_RATIO'
                ,'averageVolume': 'AVG_VOLUME'
                ,'sharesOutstanding': 'SS_SHARES_OUTSTANDING'
                ,'trailingAnnualDividendRate': 'DIVIDEND_RATE_PER_SHARE'
                ,'fiveYearAvgDividendYield': 'DIVIDEND_YIELD'
                ,'revenueGrowth': 'Y_REVENUE_GROWTH_EST'
                ,'earningsGrowth': 'Y_EARNINGS_GROWTH_EST'
                ,'country' : 'country'
               ,'sector' :  'sector' 
                ,'industry' : 'industry'
                ,'market' :  'market' 
                ,'recommendationKey':  'recommendationKey'
                ,'growth_est_next_year' : 'growth_est_next_year'
                ,'growth_est_next_5_years' : 'growth_est_next_5_years'
                
               }

Counting= 0


print(str(len(LS_TICKERS)) + ' tickets to Download')
print( 'INFO Dowload started at ' + str(datetime.datetime.now())+'\n' )



#for ticker in LS_TICKERS :
    #list_of_chunk_wk =  (' '.join(str(x) for x in list_ticker )) #Converts list to expected format
    
for ticker in LS_TICKERS:
    try:
         ##info from  yfinance
        tickers = yf.Tickers(ticker)
        Y_DICT_COLLECTION[ticker]= tickers.tickers[ticker].info
        
        ##growth from yahoo_fin
        dc_s= si.get_analysts_info(ticker)
        df_w = dc_s['Growth Estimates']
        df_w=df_w[3:5][['Growth Estimates',ticker]]
        Y_DICT_COLLECTION[ticker]['growth_est_next_year']=df_w[0:1][ticker].values[0]
        Y_DICT_COLLECTION[ticker]['growth_est_next_5_years']=df_w[1:2][ticker].values[0]
        
        print( ticker + "'s yahoo recomendation data done")
    except:
        print(ticker + ' ticker error processing INFO, Proc has  pass')
        pass
Counting+=1
print( 'List No ' + str( Counting) +  ' Download  finished')
        
print( 'INFO Dowload finished at ' + str(datetime.datetime.now())+'\n' )

DF_YAHOO_RECOMMENDATIONS_IMP= pd.DataFrame(Y_DICT_COLLECTION).T.reset_index()
DF_YAHOO_RECOMMENDATIONS_IMP=DF_YAHOO_RECOMMENDATIONS_IMP[list(columns_rename.keys())]
DF_YAHOO_RECOMMENDATIONS_IMP=DF_YAHOO_RECOMMENDATIONS_IMP.rename(columns=columns_rename)
DF_YAHOO_RECOMMENDATIONS_IMP.columns = DF_YAHOO_RECOMMENDATIONS_IMP.columns.str.lower()
DF_YAHOO_RECOMMENDATIONS_IMP['load_date'] = str(datetime.datetime.now())
DATAFRAME_COLLECTION[DF]=DF_YAHOO_RECOMMENDATIONS_IMP


        
        





2 tickets to Download
INFO Dowload started at 2021-12-31 13:36:02.020530

ABMD's yahoo recomendation data done
ABT's yahoo recomendation data done
List No 1 Download  finished
INFO Dowload finished at 2021-12-31 13:36:07.882607



In [8]:
  dc_s= si.get_analysts_info('AMZN')

In [9]:
dc_s

{'Earnings Estimate':   Earnings Estimate  Current Qtr. (Dec 2021)  Next Qtr. (Mar 2022)  \
 0   No. of Analysts                    40.00                 28.00   
 1     Avg. Estimate                     3.75                 10.87   
 2      Low Estimate                     0.97                  5.70   
 3     High Estimate                     7.04                 22.98   
 4      Year Ago EPS                    14.09                 15.79   
 
    Current Year (2021)  Next Year (2022)  
 0                46.00             47.00  
 1                40.98             51.59  
 2                38.00             32.15  
 3                45.80             73.60  
 4                41.83             40.98  ,
 'Revenue Estimate':           Revenue Estimate Current Qtr. (Dec 2021) Next Qtr. (Mar 2022)  \
 0          No. of Analysts                      40                   28   
 1            Avg. Estimate                 137.75B              121.51B   
 2             Low Estimate           

In [7]:
DF_YAHOO_RECOMMENDATIONS_IMP

,ticker,name,market_cap,y_price_1y_est,y_rank,y_rank_analyst_num,profit_margins,eps,pe_ratio,avg_volume,...,y_revenue_growth_est,y_earnings_growth_est,country,sector,industry,market,recommendationkey,growth_est_next_year,growth_est_next_5_years,load_date
0,ABMD,"ABIOMED, Inc.",16472824832,379.8,2.5,5,0.15319,3.257,111.16365,242546,...,0.183,-0.088,United States,Healthcare,Medical Devices,us_market,buy,19.30%,10.48%,2021-12-31 13:36:07.888577
1,ABT,Abbott Laboratories,249328893952,142.68,2,20,0.17122,4.03,34.98759,5339682,...,0.234,0.696,United States,Healthcare,Medical Devices,us_market,buy,-6.30%,13.12%,2021-12-31 13:36:07.888577
